In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

%matplotlib inline


nDivisionsXY = 175
sLength = 87.5
lLength = 200

def calcEscapeEnergy(eDepFileTitle, rootFileTitle, particle, energy, xinit, yinit, drawPlots:bool = False):


    #################################
    ## root file and data frame
    #################################

    tree = uproot.open(f"{rootFileTitle}_{particle}_{energy}GeV_{xinit},{yinit}.root")["Eecs"]
    df = tree.arrays(library="np")
 
    treeEdep = uproot.open(f"{rootFileTitle}_{particle}_{energy}GeV_{xinit},{yinit}.root")["Edep"]
    eDepEventWise = treeEdep.arrays(["eDep"], library="np")["eDep"]

    eDepMean = np.mean(eDepEventWise)
    eDepRMS = np.std(eDepEventWise)

    number_of_events = eDepEventWise.size

    #################################
    ## eDep File & eDep mesh
    #################################

    fileEdep = open(f"{eDepFileTitle}_{particle}_{energy}GeV_Box_{xinit},{yinit}.txt")

    mesh = np.zeros((nDivisionsXY, nDivisionsXY))

    fileEdep.readline() # skip 3 firts lines
    fileEdep.readline()
    fileEdep.readline()

    for line in fileEdep:
        lineSplit = line.split(",")

        ix = int(lineSplit[0])
        iy = int(lineSplit[1])

        eDep = float(lineSplit[3])

        mesh[ix, iy] += eDep

    mesh/=number_of_events

    #################################
    ## Variables to return
    #################################    


    ## Side escape
    sideEscapeSelect = np.fabs(df['Zpos']) < lLength-0.01
    eEscSide = df["escE"][sideEscapeSelect]
    evntIDSide = df["evntID"][sideEscapeSelect]

    sideEscEventWise = np.zeros((number_of_events,))
    for i in range(eEscSide.size):
        sideEscEventWise[evntIDSide[i]] += eEscSide[i]

    sideEscMean = np.mean(sideEscEventWise)
    sideEscRMS  = np.std(sideEscEventWise)


    ## Entry escape
    entryEscapeSelect = df['Zpos'] < -lLength+0.01
    eEscEntry = df["escE"][entryEscapeSelect]
    evntIDEntry = df["evntID"][entryEscapeSelect]

    entryEscEventWise = np.zeros((number_of_events,))
    for i in range(eEscEntry.size):
        entryEscEventWise[evntIDEntry[i]] += eEscEntry[i]

    entryEscMean = np.mean(entryEscEventWise)
    entryEscRMS  = np.std(entryEscEventWise)


    ## Exit escape
    exitEscapeSelect = df['Zpos'] > lLength-0.01
    eEscExit = df["escE"][exitEscapeSelect]
    evntIDExit = df["evntID"][exitEscapeSelect]

    exitEscEventWise = np.zeros((number_of_events,))
    for i in range(eEscExit.size):
        exitEscEventWise[evntIDExit[i]] += eEscExit[i]

    exitEscMean = np.mean(exitEscEventWise)
    exitEscRMS  = np.std(exitEscEventWise)


    ## Misscounts
    miscEventWise = energy*1000-exitEscEventWise-entryEscEventWise-\
                                sideEscEventWise-eDepEventWise
    miscMean = np.mean(miscEventWise)
    miscRMS  = np.std(miscEventWise)


    ## Total escape
    totEscEventWise = energy*1000 - eDepEventWise
    totEscMean = np.mean(totEscEventWise)
    totEscRMS  = np.std(totEscEventWise)


#     plt.figure() #fixme, delete this comented scratch code
#     esc = exitEscEventWise+entryEscEventWise+sideEscEventWise
# #     selection = ( np.fabs(esc - np.mean(esc)) < 6*np.std(esc) ) * (  np.fabs(eDepEventWise - np.mean(eDepEventWise)) < 6*np.std(eDepEventWise)  )
#     selection = np.ones_like(esc, dtype=bool)
#     plt.scatter(esc[selection], miscEventWise[selection])
# #     plt.plot([0,3000], [25000, 22000], "-r")

#     print("mean side escape", sideEscMean, "RMS", sideEscRMS)
#     print("mean entry escape", entryEscMean, "RMS", entryEscRMS)
#     print("mean exit escape",  exitEscMean, "RMS", exitEscRMS)
#     print("mean energy miscount", miscMean, "RMS", miscRMS)
#     print("mean deposited energy",  eDepMean, "RMS", eDepRMS)
#     print("mean total escaped energy", totEscMean, "RMS", totEscRMS)


    if drawPlots:

        plt.figure(figsize=(2*6.4, 4.5*4.8))
        grid = plt.GridSpec(7, 2, wspace=0.2, hspace=0.9) 

        #################################
        ## eDep plots
        #################################
        plt.subplot(grid[0:2, 0])
        plt.imshow(np.transpose(mesh), extent=(-sLength, sLength, -sLength, sLength), norm=colors.LogNorm(vmin=1e-3))
        cbar = plt.colorbar()
        cbar.ax.set_ylabel("Deposited Energy [MeV]")
        plt.xlabel("x [mm]")
        plt.ylabel("y [mm]")
        plt.title(f"Energy deposition of {energy} GeV {particle}")
        ax = plt.gca()
        ax.text(0.55, 0.01, "dx = dy = {:.2f} mm".format(sLength*2/nDivisionsXY), transform=ax.transAxes, color="white")
        ax.text(0.05, 0.01, "Mean (over events)\ndeposited energy\nis {:.2f} MeV".format(eDepMean),
                        transform=ax.transAxes, color="white")


        #################################
        ## Entry escape 2D plot
        #################################
        plt.subplot(grid[0:2, 1])
        plt.hist2d(df["Xpos"][entryEscapeSelect], df["Ypos"][entryEscapeSelect], 
                weights=df["escE"][entryEscapeSelect]/number_of_events, bins=15, norm=colors.LogNorm(vmin=0.05))
        cbar = plt.colorbar()
        cbar.ax.set_ylabel("Top-escaped energy [MeV]")
        plt.xlabel("x [mm]")
        plt.ylabel("y [mm]")
        plt.title(f"Entry-escaping energy, {particle} {energy} GeV")
        ax = plt.gca()
        ax.text(0.55, 0.01, "dx = dy = {:.2f} mm".format(sLength*2/15), transform=ax.transAxes, color="white")
        ax.text(0.05, 0.01, 
                "Mean (over events)\nentry-escaped energy\nis {:.2f} MeV".format(entryEscMean),
                transform=ax.transAxes, color="white")


        #################################
        ## Exit escape 2D plot
        #################################
        plt.subplot(grid[2:4, 1])
        plt.hist2d(df["Xpos"][exitEscapeSelect], df["Ypos"][exitEscapeSelect], 
                weights=df["escE"][exitEscapeSelect]/number_of_events, bins=15, norm=colors.LogNorm(vmin=0.05))
        cbar = plt.colorbar()
        cbar.ax.set_ylabel("Bottom-escaped energy [MeV]")
        plt.xlabel("x [mm]")
        plt.ylabel("y [mm]")
        plt.title("Exit-escaping energy")
        ax = plt.gca()
        ax.text(0.55, 0.01, "dx = dy = {:.2f} mm".format(sLength*2/15), transform=ax.transAxes, color="white")
        ax.text(0.05, 0.01, 
                "Mean (over events)\nexit-escaped energy\nis {:.2f} MeV".format(exitEscMean),
                transform=ax.transAxes, color="white")


        #################################
        ## Side escape XY 2D plot
        #################################
        plt.subplot(grid[2:4, 0])
        plt.hist2d(df["Xpos"][sideEscapeSelect], df["Ypos"][sideEscapeSelect], 
                weights=df["escE"][sideEscapeSelect]/number_of_events, bins=15, norm=colors.LogNorm(vmin=0.05))
        cbar = plt.colorbar()
        plt.plot([xinit], [yinit], "+r", label=f"{particle} entry point\nx={xinit} mm\ny={yinit} mm")
        cbar.ax.set_ylabel("Side-escaped energy [MeV]")
        plt.xlabel("x [mm]")
        plt.ylabel("y [mm]")
        plt.title("Histogram of side-escaping energy")
        ax = plt.gca()
        ax.text(0.3, 0.1, "dx = dy = {:.2f} mm".format(sLength*2/15), transform=ax.transAxes)  
        ax.legend(frameon=False, loc='center', bbox_to_anchor=(0.4, 0.8))


        #################################
        ## Side escape z coordinate hist
        #################################
        plt.subplot(grid[4, 0])
        plt.hist(df["Zpos"][sideEscapeSelect] + lLength, weights=df["escE"][sideEscapeSelect]/number_of_events, bins=100)
        plt.title("Histogram of side-escaping energy")
        plt.xlabel("Depth in volume [mm]")
        plt.ylabel("$E_{dep}$ [MeV]")
        ax = plt.gca()
        ax.text(0.02, 0.85, "The bin size is {:.2f} mm.".format(2*lLength/100), 
                           transform=ax.transAxes)


        #################################
        ## Escaped tot E. event wise hist
        #################################
        plt.subplot(grid[4, 1])
        plt.hist(totEscEventWise[np.fabs(totEscEventWise - totEscMean) < 5.5*totEscRMS], bins= int( np.sqrt(number_of_events) ))
        plt.title("Event-wise histogram of total escaped energy")
        plt.xlabel("Total escaped energy [MeV]")
        ax = plt.gca()
        ax.text(0.65, 0.7, "$\mu$ = {:.2f} MeV\nRMS = {:.2f} MeV".format(totEscMean, totEscRMS), 
                transform=ax.transAxes)


        #################################
        ## Side-escape event wise hist
        #################################
        plt.subplot(grid[5, 0])
        plt.hist(sideEscEventWise[np.fabs(sideEscEventWise - sideEscMean) < 5.5*sideEscRMS], bins= int( np.sqrt(number_of_events) ))
        plt.title("Event-wise histogram of side-escaped energy")
        plt.xlabel("Side-escaped energy [MeV]")
        ax = plt.gca()
        ax.text(0.65, 0.7, "$\mu$ = {:.2f} MeV\nRMS = {:.2f} MeV".format(sideEscMean, sideEscRMS), 
                transform=ax.transAxes)

        #################################
        ## Entry-escape event wise hist
        #################################
        plt.subplot(grid[5, 1])
        plt.hist(entryEscEventWise[np.fabs(entryEscEventWise - entryEscMean) < 5.5*entryEscRMS], bins= int( np.sqrt(number_of_events) ))
        plt.title("Event-wise histogram of entry-escaped energy")
        plt.xlabel("Entry-escaped energy [MeV]")
        ax = plt.gca()
        ax.text(0.65, 0.7, "$\mu$ = {:.2f} MeV\nRMS = {:.2f} MeV".format(entryEscMean, entryEscRMS), 
                transform=ax.transAxes)

        #################################
        ## Exit-escape event wise hist
        #################################
        plt.subplot(grid[6, 1])
        plt.hist(exitEscEventWise[np.fabs(exitEscEventWise - exitEscMean) < 5.5*exitEscRMS], bins= int( np.sqrt(number_of_events) ))
        plt.title("Event-wise histogram of exit-escaped energy")
        plt.xlabel("Exit-escaped energy [MeV]")
        ax = plt.gca()
        ax.text(0.65, 0.7, "$\mu$ = {:.2f} MeV\nRMS = {:.2f} MeV".format(exitEscMean, exitEscRMS), 
                transform=ax.transAxes)

        # #################################
        # ## E. misscount event wise hist
        # #################################
        # plt.subplot(grid[6, 0])
        # plt.hist(miscEventWise[np.fabs(miscEventWise - miscMean) < 5.5*miscRMS], bins= int( np.sqrt(number_of_events) ))
        # plt.title("Event-wise histogram of energy misscounts")
        # plt.xlabel("Energy misscounts [MeV]")
        # ax = plt.gca()
        # ax.text(0.65, 0.7, "$\mu$ = {:.2f} MeV\nRMS = {:.2f} MeV".format(miscMean, miscRMS), 
        #         transform=ax.transAxes)

        plt.savefig(f"plots/Energy_escaping_Analysis_{energy}_GeV_{particle}_x={xinit}_y={yinit}.png")


    results = {
        "totEsc": {"mean": totEscMean, "rms": totEscRMS},
        "misc": {"mean": miscMean, "rms": miscRMS},
        "exitEsc": {"mean": exitEscMean, "rms": exitEscRMS},
        "entryEsc": {"mean": entryEscMean, "rms": entryEscRMS},
        "sideEsc": {"mean": sideEscMean, "rms": sideEscRMS},
        "eDep": {"mean": eDepMean, "rms": eDepRMS}
    }

    return results

In [ ]:
position = [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84]

# position = [0, 63]

totEscMean = []
totEscRMS = []

sideEscMean = []
sideEscRMS = []

entryEscMean = []
entryEscRMS = []

exitEscMean = []
exitEscRMS = []

miscMean = []
miscRMS = []

# eDepMean = []
# eDepMean = []

for x in position:
    results = calcEscapeEnergy("build/eDep","build/eEsc", "gamma", 25, x, 0, True)

    # results = calcEscapeEnergy("build/eDep","build/eEsc", "gamma", 25, x, 0, False)
    # print("x =", x, "Escape energy rms", results['totEsc']["rms"],"resolution", results['totEsc']["rms"]/25000*100, "%")

    totEscMean.append(results["totEsc"]["mean"])
    totEscRMS.append(results["totEsc"]["rms"])

    sideEscMean.append(results["sideEsc"]["mean"])
    sideEscRMS.append(results["sideEsc"]["rms"])

    entryEscMean.append(results["entryEsc"]["mean"])
    entryEscRMS.append(results["entryEsc"]["rms"])

    exitEscMean.append(results["exitEsc"]["mean"])
    exitEscRMS.append(results["exitEsc"]["rms"])

    miscMean.append(results["misc"]["mean"])
    miscRMS.append(results["misc"]["rms"])

In [ ]:
position = np.array(position)

totEscMean   = np.array(totEscMean  )/1000
totEscRMS    = np.array(totEscRMS   )/1000

sideEscMean  = np.array(sideEscMean )/1000
sideEscRMS   = np.array(sideEscRMS  )/1000

entryEscMean = np.array(entryEscMean)/1000
entryEscRMS  = np.array(entryEscRMS )/1000

exitEscMean  = np.array(exitEscMean )/1000
exitEscRMS   = np.array(exitEscRMS  )/1000 

miscMean     = np.array(miscMean    )/1000
miscRMS      = np.array(miscRMS     )/1000

In [ ]:
plt.figure(figsize=(2*6.4, 2*4.8))
plt.errorbar(position + 0.25, totEscMean,   yerr=totEscRMS,   fmt='.', capsize=3, label="Total escaped energy")
plt.errorbar(position + 0.5, sideEscMean,  yerr=sideEscRMS,  fmt='.', capsize=3, label="Side-escaped energy")
plt.errorbar(position - 0.25, entryEscMean, yerr=entryEscRMS, fmt='.', capsize=3, label="Entry-escaped energy")
plt.errorbar(position - 0.5, exitEscMean,  yerr=exitEscRMS,  fmt='.', capsize=3, label="Exit-escaped energy")
# plt.errorbar(position, miscMean,     yerr=miscRMS,     fmt='.', capsize=3, label="Energy misscounts")

plt.ylabel("Escaped energy [GeV]")
plt.xlabel("Position of the hit [mm]")
plt.grid()
plt.legend()